In [1]:
import torch
from torch import nn
import data_utils
import eval
import auxiliary
from auxiliary import Accumulator

ModuleNotFoundError: No module named 'eval'

# Load data

In [ ]:
#Initialize data loaders
batch_size = 256
data_loaders = data_utils.load_financial(batch_size)

# Model

In [ ]:
# Hyperparameters
wd = 0.0005
lr = 0.1

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

    def forward(self,x):
        return x

# Intialization

In [ ]:
model = Net()

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear: # by checking thr type we can init different layers in different ways
        torch.nn.init.xavier_uniform_(m.weight)    

model.apply(init_weights);

# Loss and optimizer


In [ ]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), weight_decay=wd, lr=lr)

# Training

In [ ]:
def train_epoch(net, data, optimizer, batch_size=256, num_outputs=10):  
    """The training function for one epoch."""
    # Set the model to training mode
    if isinstance(net, torch.nn.Module):
        net.train()

    # Metrics
    train_metric = Accumulator(3)
    test_metric = Accumulator(2)
        
    # Go over each fold
    for fold in data:
        train_iter = fold[0]  
        test_iter = fold[1]
        #Train
        for X in train_iter:
            y_hat = net(X)
            y = None # no ground truth / label
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
            train_metric.add(float(l) * len(y), eval.accuracy(y_hat, y), y.size().numel())
        #Evaluate
        for X in test_iter:
            for _, (X, y) in enumerate(test_iter):
                test_metric.add(eval.accuracy(net(X), y), y.numel())
    
    # Return training loss, training accuracy, and test accuracy
    return train_metric[0] / train_metric[2], train_metric[1] / train_metric[2], test_metric[0] / test_metric[1]

In [ ]:
def train(net, data, loss, num_epochs, updater): 
    animator = auxiliary.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0.0, 1.0],legend=['train loss', 'train acc', 'test acc'])
    for epoch in range(num_epochs):
        metrics = train_epoch(net, data, loss, updater)
        animator.add(epoch + 1, metrics)
    train_loss, train_acc, test_acc = metrics
    print(train_loss, train_acc, test_acc)
    assert train_loss < 0.5, train_loss
    assert train_acc <= 1 and train_acc > 0.7, train_acc
    assert test_acc <= 1 and test_acc > 0.7, test_acc

# Execution

In [ ]:
num_epochs = 30
train(model, data_loaders, loss, num_epochs, optimizer)